In [1]:
import os
import socket
from pyspark import SparkConf, SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import udf, length, when, col, lit, broadcast
from pyspark.sql.types import BooleanType, IntegerType, LongType, StringType, ArrayType, FloatType, StructType, StructField
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
from pyspark import StorageLevel
from jinja2 import Environment, FileSystemLoader

from pyspark.sql.functions import * # count, desc, size, explode, arrays_zip, regexp_extract

In [2]:
from emoji import get_emoji_regexp, unicode_codes
import re

In [3]:
# setting constants
APP_NAME = "YOUR_APP_NAME"
NORMALIZED_APP_NAME = APP_NAME.replace('/', '_').replace(':', '_')

APPS_TMP_DIR = os.path.join(os.getcwd(), "tmp")
APPS_CONF_DIR = os.path.join(os.getcwd(), "conf")
APPS_LOGS_DIR = os.path.join(os.getcwd(), "logs")
LOG4J_PROP_FILE = os.path.join(APPS_CONF_DIR, "pyspark-log4j-{}.properties".format(NORMALIZED_APP_NAME))
LOG_FILE = os.path.join(APPS_LOGS_DIR, 'pyspark-{}.log'.format(NORMALIZED_APP_NAME))
EXTRA_JAVA_OPTIONS = "-Dlog4j.configuration=file://{} -Dspark.hadoop.dfs.replication=1 -Dhttps.protocols=TLSv1.0,TLSv1.1,TLSv1.2,TLSv1.3"\
    .format(LOG4J_PROP_FILE)

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [4]:
# preparing configuration files from templates
for directory in [APPS_CONF_DIR, APPS_LOGS_DIR, APPS_TMP_DIR]:
    if not os.path.exists(directory):
        os.makedirs(directory)

env = Environment(loader=FileSystemLoader('/opt'))
template = env.get_template("pyspark_log4j.properties.template")
template\
    .stream(logfile=LOG_FILE)\
    .dump(LOG4J_PROP_FILE)

In [5]:
# run spark
spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master("k8s://https://10.32.7.103:6443")\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.ui.port", "4040")\
    .config("spark.kubernetes.memoryOverheadFactor", "0.6")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "5")\
    .config("spark.executor.cores", '4')\
    .config("spark.executor.memory", "5g")\
    .config("spark.memory.fraction", "0.6")\
    .config("spark.memory.storageFraction", "0.5")\
    .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
    .config("spark.driver.extraJavaOptions", EXTRA_JAVA_OPTIONS)\
    .config("spark.kubernetes.namespace", "szhumabaev-307617")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.container.image.pullPolicy", "Always")\
    .config("spark.kubernetes.container.image", "node03.st:5000/spark-executor:szhumabaev-307617")\
    .config("spark.kubernetes.executor.deleteOnTermination", "true")\
    .config("spark.local.dir", "/tmp/spark")\
    .getOrCreate()

In [6]:
spark.version

'3.1.1'

## Task 1

Find the top 20 posts in the group: 
* by likes; 
* by comments; 
* by reposts.

In [7]:
posts_df = spark.read.json("/shared/bigdata20/posts_api.json")\
    .select(
        col("id").alias("post_id"), col("likes.count").alias("cnt_likes"), 
        col("comments.count").alias("cnt_comments"), col("reposts.count").alias("cnt_reposts"), "text"
    )

print(posts_df.count())  # 18 430

18430


In [8]:
# by likes
posts_df.select("post_id", "cnt_likes").orderBy(desc("cnt_likes")).show()

+-------+---------+
|post_id|cnt_likes|
+-------+---------+
|  32022|     1637|
|  35068|     1629|
|  17492|     1516|
|  18526|     1026|
|  19552|      955|
|  41468|      952|
|  19419|      868|
|  29046|      824|
|  32546|      786|
|  24085|      765|
|  40180|      759|
|  33658|      708|
|  13532|      633|
|  40842|      631|
|  35117|      588|
|  17014|      581|
|  19583|      553|
|  19809|      552|
|  27455|      550|
|  11999|      549|
+-------+---------+
only showing top 20 rows



In [9]:
# by comments
posts_df.select("post_id", "cnt_comments").orderBy(desc("cnt_comments")).show()

+-------+------------+
|post_id|cnt_comments|
+-------+------------+
|  24085|         850|
|  22540|         250|
|  27722|         192|
|   8285|         148|
|  26860|         113|
|  13571|         107|
|  39294|         104|
|  36680|          96|
|  26006|          92|
|  41739|          92|
|  12426|          91|
|  21499|          88|
|  39163|          83|
|  39407|          83|
|  11267|          81|
|  31548|          80|
|  11158|          70|
|  39082|          67|
|  14602|          61|
|  12687|          61|
+-------+------------+
only showing top 20 rows



In [10]:
# by reposts
posts_df.select("post_id", "cnt_reposts").orderBy(desc("cnt_reposts")).show()

+-------+-----------+
|post_id|cnt_reposts|
+-------+-----------+
|  17492|        334|
|  19552|        246|
|  32022|        210|
|  11842|        129|
|  19419|        126|
|  13532|        110|
|  17014|        105|
|  35068|        101|
|  41266|         92|
|  12593|         90|
|  29046|         87|
|  41468|         85|
|  11999|         85|
|  19809|         84|
|  17167|         81|
|  10833|         78|
|  18543|         77|
|  16596|         76|
|  18156|         74|
|  37262|         71|
+-------+-----------+
only showing top 20 rows



## Task 2

Find the top 20 users by (a) likes and (b) reposts they have made (to trace reposts use "*copy_history*" field)

In [11]:
# by likes

spark.read.parquet("/shared/bigdata20/posts_likes.parquet")\
    .groupBy(col("likerId").alias("user_id")).agg(count("itemId").alias("cnt_posts"))\
    .orderBy(desc("cnt_posts")).show()

+---------+---------+
|  user_id|cnt_posts|
+---------+---------+
|  2070090|     4801|
|  2397858|     2055|
|  1475301|     1829|
|    18239|     1569|
|   546612|     1245|
|     6371|      907|
|  1841959|      746|
| 78440957|      709|
|   120248|      699|
| 40981497|      611|
|    22158|      553|
|207628162|      548|
|329377723|      504|
| 76071304|      474|
| 14805173|      440|
|   317799|      385|
| 56355640|      375|
| 52042971|      338|
|  7437271|      336|
|136506644|      335|
+---------+---------+
only showing top 20 rows



In [12]:
# by reposts

spark.read.json("/shared/bigdata20/followers_posts_api_final.json")\
    .where("copy_history is not null")\
    .select(
        col("owner_id").alias("user_id"),
        col("copy_history.id").getItem(0).alias("src_post_id"),
        col("copy_history.owner_id").getItem(0).alias("src_owner_id")
    ).groupBy("user_id").agg(count("src_post_id").alias("cnt_reposts"))\
    .orderBy(desc("cnt_reposts")).show()

+---------+-----------+
|  user_id|cnt_reposts|
+---------+-----------+
|  2547211|      37742|
|357231922|      23349|
|168543860|      18429|
| 25646344|      11122|
|176861294|       9022|
|524656784|       7242|
|    29840|       7164|
|143207077|       7161|
|141687240|       6804|
|459339006|       6741|
|514384760|       6570|
|483715951|       6052|
|445159771|       5808|
|451211328|       5646|
|426396104|       5533|
|  8325325|       5532|
|452280411|       5458|
|464220898|       5318|
|440454268|       5304|
|461319529|       5240|
+---------+-----------+
only showing top 20 rows



## Task 3

Get reposts of the original posts of the itmo group (*posts.json*) from user posts 
(the result should be similar to (*group_post_id*, *Array(user_post_ids)*))

In [13]:
spark.read.json("/shared/bigdata20/followers_posts_api_final.json")\
    .where("copy_history is not null")\
    .select(
        col("id").alias("user_post_id"), 
        col("copy_history.id").getItem(0).alias("group_post_id"),
        col("copy_history.owner_id").getItem(0).alias("post_owner_id")
    ).join(
        spark.read.json("/shared/bigdata20/posts_api.json")
            .select(col("id").alias("group_post_id"), col("owner_id").alias("post_owner_id")), 
        ["post_owner_id", "group_post_id"]
    ).groupBy("group_post_id").agg(collect_list("user_post_id").alias("user_post_ids"))\
    .withColumn("cnt_reposts", size("user_post_ids"))\
    .orderBy(desc("cnt_reposts"), "group_post_id").show()

+-------------+--------------------+-----------+
|group_post_id|       user_post_ids|cnt_reposts|
+-------------+--------------------+-----------+
|        41266|[1150, 235, 452, ...|         30|
|        41468|[1931, 5408, 440,...|         25|
|        42482|[4205, 713, 1542,...|         10|
|        40090|[463, 1325, 349, ...|          9|
|        38740|[1060, 1133, 185,...|          8|
|        39259|[10810, 10683, 12...|          8|
|        41207|[2960, 10610, 128...|          6|
|        41546|[666, 3908, 7005,...|          6|
|        41721|[3810, 2801, 1089...|          6|
|        38963|[3720, 6403, 814,...|          5|
|        39682|[384, 159, 600, 3...|          5|
|        41506|[397, 398, 12823,...|          5|
|        38915|[1186, 18054, 448...|          4|
|        39294|[4516, 2319, 4718...|          4|
|        39515|[7390, 443, 2321,...|          4|
|        39686|[182, 3370, 4741,...|          4|
|        41108|[182, 366, 10483,...|          4|
|        41708|[1010

## Task 4

Find emoticons in posts, calculate their overall count, 
frequency (number of posts they can be found in) and average count per post. 

* print top 10 most popular emoticons;
* print top 5 emoticons which have the greatest difference between their overall count and frequency; 
* print top 5 emoticons with average count per post.


Note: you can use external libraries or predefined emoticon lists

Note: build an UDF from your code for emoticon checking to use it with pyspark

In [14]:
reg_exp = get_emoji_regexp()
unicode_emoji = unicode_codes.UNICODE_EMOJI["en"]

@udf(returnType=ArrayType(StringType()))
def emoji_udf(text_col):
    _entities = []
    for match in reg_exp.finditer(text_col):
        _entities.append(match.group())
    return _entities

@udf
def demojize_udf(text_col):    
    def replace(match):
        val = unicode_emoji.get(match.group(0), match.group(0))
        return ":" + val[1:-1] + ":"

    return re.sub(u'\ufe0f', '', (reg_exp.sub(replace, text_col)))

In [15]:
emoji_df = spark.read.json("/shared/bigdata20/posts_api.json")\
    .where("text <> ''").select("id", "text")\
    .withColumn("emoticons", emoji_udf(col("text")))\
    .where("size(emoticons) > 0")\
    .select("id", lit(demojize_udf(col("emoji"))).alias("demojize"), explode("emoticons").alias("emoji"))\
    .groupBy("demojize", "emoji").agg(
        count("id").alias("cnt_all"),
        countDistinct("id").alias("frequency"),
    ).withColumn("avg_cnt_per_post", round(col("cnt_all") / col("frequency"), 3))\
    .withColumn("diff", col("cnt_all") - col("frequency"))

In [16]:
# print top 10 most popular emoticons

emoji_df.select("demojize", "emoji", "cnt_all")\
    .orderBy(desc("cnt_all")).limit(10).toPandas()

,demojize,emoji,cnt_all
0,:fire:,🔥,76
1,:high_voltage:,⚡,68
2,:round_pushpin:,📍,63
3,:rocket:,🚀,50
4,:red_heart:,❤,47
5,:red_exclamation_mark:,❗,45
6,:down_arrow:,⬇,40
7,:blue_circle:,🔵,40
8,:check_mark:,✔,40
9,:blue_heart:,💙,38


In [17]:
# print top 5 emoticons which have the greatest difference between their overall count and frequency

emoji_df.select("demojize", "emoji", "diff")\
    .orderBy(desc("diff")).limit(5).toPandas()

,demojize,emoji,diff
0,:check_mark:,✔,31
1,:red_exclamation_mark:,❗,25
2,:round_pushpin:,📍,24
3,:down_arrow:,⬇,24
4,:high_voltage:,⚡,23


In [18]:
# print top 5 emoticons with average count per post

emoji_df.select("demojize", "emoji", "avg_cnt_per_post")\
    .orderBy(desc("avg_cnt_per_post")).limit(5).toPandas()

,demojize,emoji,avg_cnt_per_post
0,:check_mark:,✔,4.444
1,:small_blue_diamond:,🔹,4.286
2,:black_small_square:,▪,4.200
3,:black_nib:,✒,3.500
4,:oncoming_taxi:,🚖,3.000


## Task 5

Probable *"fans"*. Find for each user the top 10 other users whose posts this user likes.

In [19]:
fans_df = spark.read.parquet("/shared/bigdata20/followers_posts_likes.parquet")\
    .withColumn("cnt", count("itemId").over(Window.partitionBy("ownerId", "likerId")))\
    .select("ownerId", "likerId", "cnt").distinct()\
    .withColumn("top", row_number().over(Window.partitionBy("likerId").orderBy(desc("cnt"), "ownerId")))\
    .where("top <= 10")

fans_df.orderBy(desc("likerId"), "top").show()

+---------+---------+---+---+
|  ownerId|  likerId|cnt|top|
+---------+---------+---+---+
|331271681|566943808|  2|  1|
| 23380429|566924016|  1|  1|
|  5786733|566921437|  1|  1|
|     1087|566920553|  1|  1|
|  5786733|566916469|  1|  1|
|   222923|566900723|  1|  1|
| 20224097|566895285|  1|  1|
| 62234805|566895285|  1|  2|
|  1536597|566892214|  1|  1|
|121780041|566892214|  1|  2|
|180124822|566890516|  1|  1|
|  3581627|566882374|  1|  1|
|   479142|566874253|  1|  1|
|187877260|566868012|  1|  1|
|534742309|566862000|  2|  1|
| 16861388|566856410|  1|  1|
|276887938|566834379|  1|  1|
|   368107|566834334|  1|  1|
|220846414|566833175|  1|  1|
|139509646|566816578|  1|  1|
+---------+---------+---+---+
only showing top 20 rows



## Task 6

Probable friends. If two users like each other posts they may be friends. 

Find pairs of users where both users are top likers of each other.

In [20]:
user_df_1 = fans_df.where("top = 1")\
    .select(col("ownerId").alias("user_1"), col("likerId").alias("user_2")).distinct()

user_df_2 = fans_df.where("top = 1")\
    .select(col("ownerId").alias("user_2"), col("likerId").alias("user_1")).distinct()

user_df_1.join(user_df_2, ["user_1", "user_2"]).where("user_1 <> user_2")\
    .withColumn("user_l", least(col("user_1"), col("user_2")))\
    .withColumn("user_r", greatest(col("user_1"), col("user_2")))\
    .select("user_1", "user_r").show()

+---------+---------+
|   user_1|   user_r|
+---------+---------+
| 95578891|152590008|
|387540163|387540163|
| 87779884|145291328|
| 70730078| 71427292|
|291661975|291661975|
|   817770|  3420917|
|450562601|450562601|
|  6282600|  6801684|
| 20224097|101096378|
|120618620|234528324|
|327790230|327790230|
| 42247975| 42247975|
| 44770563|103229751|
| 45266920|188264983|
|229861638|229861638|
| 95356919| 95356919|
|214298116|214298116|
|320690911|320690911|
|379134974|379134974|
|470163231|470163231|
+---------+---------+
only showing top 20 rows



In [21]:
spark.stop()